In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [3]:
stats = pd.read_excel('COMBINED_NORMAL_ADVANCED_STATS_1984-85_to_2019-20_ver2.xlsx', 0)
salaries = pd.read_excel('salaries_1985to2020_final.xlsx')
salary_caps = pd.read_excel('salary_caps.xlsx')

In [4]:
file2 = pd.ExcelFile("2019_20_FINAL_normal_advanced_stats_V2.xlsx")
data_2020 = pd.read_excel(file2, 0)

In [5]:
stats = stats.drop(columns=['slug', 'key'])
data_2020 = data_2020.drop(columns=['Unnamed: 0'])

In [6]:
all([stats.columns.values[i]==data_2020.columns.values[i] for i in range(len(stats.columns.values))])

True

In [7]:
stats = stats.rename(columns = {'Year': 'season'})
salaries = salaries.rename(columns = {"names":'name'})

In [8]:
train = stats.merge(salaries, on = ['name', 'season'])

In [9]:
salary_caps = salary_caps.drop(columns=['Unnamed: 0'])
salary_caps.head()

,season,salary_cap
0,2027-28,167513000
1,2026-27,159536000
2,2025-26,151939000
3,2024-25,144704000
4,2023-24,137813000


In [10]:
train_with_sc = train.merge(salary_caps, on = ['season'])

In [12]:
individual_salaries = train_with_sc['salary']
salary_caps1 = train_with_sc['salary_cap']

In [13]:
cap_usage = individual_salaries/salary_caps1

In [14]:
train_with_sc['cap_usage'] = cap_usage

In [15]:
train_with_sc[['true_shooting_percentage', 'win_shares', 'cap_usage']]

,true_shooting_percentage,win_shares,cap_usage
0,0.628,11.2,0.425000
1,0.578,6.8,0.138889
2,0.563,7.2,0.222222
3,0.570,6.6,0.111111
4,0.524,4.6,0.058056
...,...,...,...
14870,0.611,3.9,0.132799
14871,0.000,0.0,0.018985
14872,0.640,0.3,0.018985
14873,0.633,1.8,0.015166


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

### BUILD MODEL HERE

In [55]:
def model_builder(df, features, response = 'cap_usage'):
    """df: dataframe with stats, salaries, and cap usage
    features: is a list of metrics for our regression model to predict salary usage."""
    df_features = df[features]
    X = df_features
    #X = StandardScaler().fit_transform(X)
    y = df[[response]]
    reg = LinearRegression().fit(X, y)
    print('The coefficients are {}'.format(reg.coef_))
    print("the R^2 score is ", reg.score(X, y))
    return reg

def predict_player(test_df, model, name_of_player, features):
    """player_features: must be the same metrics used to build the model in a df"""
    #parameters = model.get_params
    print(model.get_params)
    player_only = test_df[test_df['name'] == name_of_player]
    ready_for_test = player_only[features]
    prediction = model.predict(ready_for_test)[0][0]
    return [name_of_player, prediction], prediction

def predict_all(test_df, model, features):
    test_df_clean = test_df.dropna()
    players = test_df_clean['name'].values
    ready_for_test = test_df_clean[features]
    predictions = model.predict(ready_for_test)
    print('The mean of cap usage predictions for all players is {}'.format(np.mean(predictions)))
    #print(players.shape, predictions.shape)
    return [[players[i], predictions[i, 0]] for i in range(len(players))], [predictions[i, 0] for i in range(len(players))]

In [56]:
def do_all(train_df, test_df, features, response = 'cap_usage', predict_all_bool = False, player_names = ['Steven Adams']):
    model = model_builder(train_df, features, response)
    if predict_all_bool:
        return predict_all(test_df, model, features)
    predictions = []
    values = []
    for player in player_names:
        value = predict_player(test_df, model, player, features)
        predictions.append(value[0])
        values.append(value[1])
    return predictions, values

In [57]:
rishab_and_alvin_best = ['value_over_replacement_player', 'points', 'player_efficiency_rating', 'box_plus_minus', 'steal_percentage', 'age']
from_team = ['value_over_replacement_player', 'win_shares', 'player_efficiency_rating', 'minutes_played', 'box_plus_minus']

### RUN MODEL HERE

In [91]:
#test = do_all(train_with_sc, data_2020, from_team, predict_all_bool = True)
#test[1]

In [ ]:
model = model_builder(train_with_sc, ['true_shooting_percentage', 'win_shares'])
bam_adebayo = predict_2020(data_2020, model, "Bam Adebayo", ['true_shooting_percentage', 'win_shares'])

In [ ]:
predict_2020(data_2020, model, "Evan Turner", ['true_shooting_percentage', 'win_shares'])

In [ ]:
predict_all(data_2020, model, ['true_shooting_percentage', 'win_shares'])

In [ ]:
data_2020[data_2020['name'] == 'Jabari Parker'][from_team]

In [ ]:
train_with_sc.to_excel('Complete_Train_Set_With_Cap_Usage.xlsx')

In [ ]:
data_2020.to_excel('Clean_2019_2020_Data.xlsx')

In [132]:
train_with_sc.head()

,name,positions,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,...,value_over_replacement_player,season,player_id,salary,season_end,season_start,team_y,key,salary_cap,cap_usage
0,Kareem Abdul-Jabbar,CENTER,37,LOS ANGELES LAKERS,79,1.000000,33.291139,9.151899,15.278481,0.000000,...,4.8,1984-85,abdulka01,1530000,1985,1984,Los Angeles Lakers,Kareem Abdul-Jabbar1984-85,3600000,0.425000
1,Alvan Adams,POWER FORWARD,30,PHOENIX SUNS,82,0.841463,26.048780,5.804878,11.158537,0.000000,...,3.2,1984-85,adamsal01,500000,1985,1984,Phoenix Suns,Alvan Adams1984-85,3600000,0.138889
2,Mark Aguirre,SMALL FORWARD,25,DALLAS MAVERICKS,80,0.987500,33.737500,9.925000,19.612500,0.337500,...,2.6,1984-85,aguirma01,800000,1985,1984,Dallas Mavericks,Mark Aguirre1984-85,3600000,0.222222
3,Danny Ainge,SHOOTING GUARD,25,BOSTON CELTICS,75,0.973333,34.186667,5.586667,10.560000,0.200000,...,1.8,1984-85,aingeda01,400000,1985,1984,Boston Celtics,Danny Ainge1984-85,3600000,0.111111
4,John Bagley,POINT GUARD,24,CLEVELAND CAVALIERS,81,0.802469,29.641975,4.172840,8.555556,0.037037,...,2.2,1984-85,baglejo01,209000,1985,1984,Cleveland Cavaliers,John Bagley1984-85,3600000,0.058056


The coefficients are [[ 0.0246062  -0.00489452  0.00370078  0.00402703 -0.00404144]]
0.3712097270445499
The mean of cap usage predictions is 0.0765147105661234


0.0038598968700949343

In [123]:
compare_results = pd.DataFrame(names, columns=['player_name', 'predicted_cap_usage'])
compare_results['actual_cap_usage'] = X_test['cap_usage'].reset_index()['cap_usage']
compare_results['error'] = compare_results['predicted_cap_usage'] - compare_results['actual_cap_usage']
compare_results['error_squared'] = compare_results['error']*compare_results['error']
np.mean(compare_results['error_squared'])

0.003805952035423104

In [129]:
compare_results.sort_values(by=['error_squared']).to_excel('2017-18_predictions_model1.xlsx')

# Tune Model Parameters

In [214]:
#first splits the dataset, filters out data by year and minutes_played (per-48) and trains/runs linear_regression model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

minutes = 10
year = 2010


features = from_team + ['cap_usage'] + ['name']
X = train_with_sc.loc[(train_with_sc['season_start'] >= year) & (train_with_sc['minutes_played'] >= minutes)]


X_train, X_test, y_train, y_test = train_test_split(X, X[['cap_usage']], test_size=0.20, random_state=42)
names, values = do_all(X_train, X_test, features=from_team, predict_all_bool = True)
error = mean_squared_error(X_test['cap_usage'], y_pred=values)


compare_results = pd.DataFrame(names, columns=['player_name', 'predicted_cap_usage'])
compare_results['actual_cap_usage'] = X_test['cap_usage'].reset_index()['cap_usage']
compare_results['error'] = compare_results['predicted_cap_usage'] - compare_results['actual_cap_usage']
compare_results['error_squared'] = compare_results['error']*compare_results['error']
print(np.mean(compare_results['error_squared']), 'is R^2')

The coefficients are [[ 0.0246062  -0.00489452  0.00370078  0.00402703 -0.00404144]]
0.3712097270445499
The mean of cap usage predictions is 0.0765147105661234
0.0038598968700949347 is R^2


In [ ]:
compare_results.sort_values(by=['error_squared']).to_excel('2017-18_predictions_model1.xlsx')

In [165]:
#model parameter tuning version 1 -- year and minutes
features = from_team + ['cap_usage'] + ['name']
df = pd.DataFrame(columns=['year', 'minutes_played','error'])
for year in range(2000, 2016):
    for minutes in range(0, 20, 2):
        X = train_with_sc.loc[(train_with_sc['season_start'] >= year) & (train_with_sc['minutes_played'] >= minutes)]
        X_train, X_test, y_train, y_test = train_test_split(X, X[['cap_usage']], test_size=0.20, random_state=42)
        names, values = do_all(X_train, X_test, features=from_team, predict_all_bool = True)
        error = mean_squared_error(X_test['cap_usage'], y_pred=values)
        df = df.append({'year': year, 'minutes_played':minutes, 'error':error}, ignore_index=True)
df.to_excel('model1_tuning.xlsx')

The coefficients are [[ 2.42749724e-02 -5.17807921e-03  6.41314912e-04  3.75292983e-03
  -6.01869902e-05]]
0.3862734021604821
The mean of cap usage predictions is 0.07466871569736265
The coefficients are [[ 0.02553218 -0.00538429  0.00286068  0.00373487 -0.00368342]]
0.3859556253314509
The mean of cap usage predictions is 0.07555934556078059
The coefficients are [[ 0.02690401 -0.00575776  0.00324168  0.00387214 -0.00407737]]
0.3842943949538168
The mean of cap usage predictions is 0.07791521382343788
The coefficients are [[ 0.02619396 -0.00540899  0.00364775  0.00381519 -0.00442029]]
0.37915642598101273
The mean of cap usage predictions is 0.08000692839820767
The coefficients are [[ 0.02492952 -0.00588797  0.00371907  0.00404943 -0.0045678 ]]
0.34749847121319644
The mean of cap usage predictions is 0.08265135470278288
The coefficients are [[ 0.0235096  -0.00555527  0.00428817  0.00415634 -0.00476211]]
0.34302230588279525
The mean of cap usage predictions is 0.08623191559302215
The coeff

The coefficients are [[ 0.02431626 -0.0046084   0.00284618  0.0036446  -0.00358031]]
0.38288630562249615
The mean of cap usage predictions is 0.07378778064598458
The coefficients are [[ 0.02552191 -0.00519804  0.00300727  0.00375489 -0.0037105 ]]
0.3821480947663488
The mean of cap usage predictions is 0.0768046364458228
The coefficients are [[ 0.02443362 -0.00476293  0.00345109  0.00388209 -0.00440988]]
0.382332389746572
The mean of cap usage predictions is 0.07619697778178713
The coefficients are [[ 0.02288422 -0.00536247  0.00339418  0.0041597  -0.00360619]]
0.36161721880148046
The mean of cap usage predictions is 0.0814170119678042
The coefficients are [[ 0.02070108 -0.00493529  0.00381922  0.00429883 -0.0027993 ]]
0.3598501881679783
The mean of cap usage predictions is 0.08527252214838163
The coefficients are [[ 0.02006246 -0.00553764  0.00439937  0.00451479 -0.00265324]]
0.3535162413230998
The mean of cap usage predictions is 0.09241091646691438
The coefficients are [[ 0.01691708 

The coefficients are [[ 0.02525848 -0.00443978  0.00312145  0.00373552 -0.00359093]]
0.40048774572442647
The mean of cap usage predictions is 0.07294716560368703
The coefficients are [[ 0.02514922 -0.00484496  0.00365334  0.00387787 -0.00407495]]
0.3877677500599751
The mean of cap usage predictions is 0.07354622480527379
The coefficients are [[ 0.0246062  -0.00489452  0.00370078  0.00402703 -0.00404144]]
0.3712097270445499
The mean of cap usage predictions is 0.0765147105661234
The coefficients are [[ 0.02009948 -0.00334116  0.00410483  0.00397836 -0.00370235]]
0.3509503712160865
The mean of cap usage predictions is 0.07936154485744036
The coefficients are [[ 0.01964005 -0.00377151  0.00442296  0.00438329 -0.00353399]]
0.35763997228546773
The mean of cap usage predictions is 0.08801418287535241
The coefficients are [[ 0.01384648 -0.0033041   0.00454119  0.0046255  -0.00124971]]
0.3390382893349142
The mean of cap usage predictions is 0.09321665228829029
The coefficients are [[ 0.0165780

The coefficients are [[ 0.01985319 -0.00129993  0.00227135  0.00354234 -0.00227436]]
0.39725045903864065
The mean of cap usage predictions is 0.07180570743610591
The coefficients are [[ 0.01930457 -0.00116068  0.00261884  0.00401616 -0.00299077]]
0.3983740677441323
The mean of cap usage predictions is 0.06898741855308664
The coefficients are [[ 0.0198962  -0.00244995  0.00245583  0.00408305 -0.00149807]]
0.39386599657929855
The mean of cap usage predictions is 0.08144539593290513
The coefficients are [[ 0.0165822  -0.00042698  0.00298706  0.00396002 -0.00183621]]
0.3847965779723005
The mean of cap usage predictions is 0.08084624653836894
The coefficients are [[ 0.01705593 -0.0013826   0.00260826  0.00448343 -0.00140944]]
0.358515238318011
The mean of cap usage predictions is 0.08855083820524487
The coefficients are [[ 0.01688449 -0.00201427  0.00348265  0.00522362 -0.0022415 ]]
0.34451037484499236
The mean of cap usage predictions is 0.0947246698317587


In [166]:
#model parameter tuning attempt 2 - year and minutes -- training dataset is 2017-2019 data
features = from_team + ['cap_usage'] + ['name']
df = pd.DataFrame(columns=['year', 'minutes_played','error'])
for year in range(2000, 2016):
    for minutes in range(0, 20, 2):
        X = train_with_sc.loc[(train_with_sc['season_start'] >= year) & (train_with_sc['minutes_played'] >= minutes)]
        X_train = X.loc[(X['season_start'] != 2017) | (X['season_start'] != 2018)]
        X_test = X.loc[(X['season_start'] == 2017) | (X['season_start'] == 2018)]
        names, values = do_all(X_train, X_test, features=from_team, predict_all_bool = True)
        error = mean_squared_error(X_test['cap_usage'], y_pred=values)
        df = df.append({'year': year, 'minutes_played':minutes, 'error':error}, ignore_index=True)
df.to_excel('model1_tuning_v2.xlsx')

The coefficients are [[ 2.40460484e-02 -4.70090244e-03  5.60664590e-04  3.67938467e-03
  -5.06967534e-05]]
0.38188405882689347
The mean of cap usage predictions is 0.07508648275764594
The coefficients are [[ 0.02624505 -0.00556738  0.00285583  0.00378204 -0.00366389]]
0.38675136445475355
The mean of cap usage predictions is 0.07527537493061694
The coefficients are [[ 0.02624854 -0.00572319  0.00324322  0.00382408 -0.00410315]]
0.37909960311967505
The mean of cap usage predictions is 0.07685152718242325
The coefficients are [[ 0.0258839  -0.00579943  0.0035188   0.0038901  -0.00429542]]
0.36964113160514644
The mean of cap usage predictions is 0.07999917654238665
The coefficients are [[ 0.02545258 -0.00588804  0.00398988  0.00401078 -0.00476032]]
0.36016961199277964
The mean of cap usage predictions is 0.08148232430216995
The coefficients are [[ 0.02442579 -0.00591796  0.00435967  0.00413492 -0.00475505]]
0.34665956960797406
The mean of cap usage predictions is 0.08440615358940104
The co

The coefficients are [[ 0.02514946 -0.00513978  0.00304677  0.00373933 -0.00375432]]
0.3821121768678263
The mean of cap usage predictions is 0.07610833464294671
The coefficients are [[ 0.0246716  -0.00515812  0.00334076  0.00387959 -0.00406389]]
0.3734958474226824
The mean of cap usage predictions is 0.07748829280887762
The coefficients are [[ 0.02347468 -0.00513234  0.00364177  0.00404731 -0.00399757]]
0.361261097167653
The mean of cap usage predictions is 0.08018977462842729
The coefficients are [[ 0.0215382  -0.00508016  0.00399538  0.0042585  -0.00358078]]
0.35146267269384
The mean of cap usage predictions is 0.08206118496855544
The coefficients are [[ 0.01970445 -0.0050298   0.0044223   0.00445976 -0.00322929]]
0.3408742902225288
The mean of cap usage predictions is 0.085315950290548
The coefficients are [[ 0.01646027 -0.00483749  0.00477403  0.00474383 -0.00203822]]
0.3265760602169314
The mean of cap usage predictions is 0.09133641093146591
The coefficients are [[ 0.01560899 -0.0

The coefficients are [[ 0.02262218 -0.00388607  0.00371377  0.00400784 -0.00401434]]
0.3741563665843096
The mean of cap usage predictions is 0.07759201605058108
The coefficients are [[ 0.02111481 -0.00389655  0.00408048  0.0041786  -0.00381163]]
0.3637203713413041
The mean of cap usage predictions is 0.07984520785264468
The coefficients are [[ 0.02007376 -0.00388518  0.00450113  0.00432101 -0.00388019]]
0.35130328135199684
The mean of cap usage predictions is 0.08354353525500573
The coefficients are [[ 0.01600589 -0.0035571   0.00478661  0.00472231 -0.00246003]]
0.3405692039572613
The mean of cap usage predictions is 0.08941820817139315
The coefficients are [[ 0.01510538 -0.00369384  0.00494516  0.00506258 -0.0021054 ]]
0.32191198705408364
The mean of cap usage predictions is 0.09412347457441037
The coefficients are [[ 0.02409553 -0.00324619  0.00256364  0.00339694 -0.00295992]]
0.4101440569874989
The mean of cap usage predictions is 0.06795598160835378
The coefficients are [[ 0.024333

The mean of cap usage predictions is 0.07709189533635327
The coefficients are [[ 0.01831719 -0.0016199   0.00274179  0.0041252  -0.00170829]]
0.3800788940980473
The mean of cap usage predictions is 0.08119435503666972
The coefficients are [[ 0.0162919  -0.00156771  0.00299307  0.00461263 -0.00121888]]
0.37238749139479876
The mean of cap usage predictions is 0.08710139185101343
The coefficients are [[ 0.01563723 -0.00169802  0.00309559  0.00496987 -0.00093757]]
0.3556677970589981
The mean of cap usage predictions is 0.09199562875451037


# Tune Model Features

In [220]:
%%time
import itertools
#first choose the candidate features and the number of features, 
#then loop over every combination of them and train/run the model to calculate the error
num_features_to_choose = 3
features = ['player_efficiency_rating', 'value_over_replacement_player', 'true_shooting_percentage', 'points', 'win_shares',
           'three_point_attempt_rate', 'usage_percentage', 'assists', 'total_rebound_percentage','box_plus_minus',
           'steal_percentage','block_percentage', 'turnover_percentage', 'free_throw_attempt_rate', 'attempted_field_goals', 'made_three_point_field_goals'
           ,'minutes_played','steals',
 'blocks'] 


options = list(itertools.combinations(features, num_features_to_choose))
X = train_with_sc.loc[(train_with_sc['season_start'] >= 2010)]

columns = ['option' + str(i+1) for i in range(num_features_to_choose)] 
columns.append('error')
df = pd.DataFrame(columns=columns)
X_train, X_test, y_train, y_test = train_test_split(X, X[['cap_usage']], test_size=0.20, random_state=42)

for option in options:
    names, values = do_all(X_train, X_test, features=list(option), predict_all_bool = True)
    error = mean_squared_error(X_test['cap_usage'], y_pred=values)
    new_row = {columns[i]:option[i] for i in range(len(option))}
    new_row['error'] = error
    df = df.append(new_row, ignore_index=True)
df.to_excel('model1_feature_optimization_{0}features_from2010.xlsx'.format(num_features_to_choose))

The coefficients are [[ 0.00249526  0.02804942 -0.05960774]]
0.31093701486948466
The mean of cap usage predictions is 0.06869603483118757
The coefficients are [[-0.00021751  0.01246233  0.00645809]]
0.4058865996692663
The mean of cap usage predictions is 0.06894265227346394
The coefficients are [[0.00130901 0.01522773 0.0071007 ]]
0.3177861956518466
The mean of cap usage predictions is 0.06889200056316945
The coefficients are [[ 0.00143293  0.02949467 -0.02088423]]
0.3109237209116238
The mean of cap usage predictions is 0.0685741283256952
The coefficients are [[0.00114561 0.02726454 0.00177347]]
0.3204301888998533
The mean of cap usage predictions is 0.06892407734273794
The coefficients are [[0.00145894 0.0234914  0.00745001]]
0.3288337547808756
The mean of cap usage predictions is 0.0685287948851831
The coefficients are [[0.00144529 0.02916248 0.00083205]]
0.31013113376931944
The mean of cap usage predictions is 0.06855726681356071
The coefficients are [[ 0.00316448  0.0293384  -0.002

The coefficients are [[ 0.00304666 -0.02510514  0.01438858]]
0.2556824810066982
The mean of cap usage predictions is 0.06823404434109527
The coefficients are [[0.00456698 0.00056801 0.00063892]]
0.15915821191968438
The mean of cap usage predictions is 0.06839160983576113
The coefficients are [[ 0.00560719 -0.00151925 -0.00135283]]
0.15919170842849695
The mean of cap usage predictions is 0.06839337538924933
The coefficients are [[ 0.004842   -0.00561361 -0.00559588]]
0.16349423801975171
The mean of cap usage predictions is 0.06839685904804928
The coefficients are [[ 0.00472556 -0.01307684 -0.00200447]]
0.15986916505313975
The mean of cap usage predictions is 0.06845988767907478
The coefficients are [[ 4.64396457e-03 -6.91307888e-03 -1.91531444e-05]]
0.15796374883525943
The mean of cap usage predictions is 0.06842812526604132
The coefficients are [[ 0.00476969 -0.01053938 -0.01590394]]
0.16015199670564884
The mean of cap usage predictions is 0.06859644635302321
The coefficients are [[ 0.

0.37752197160278933
The mean of cap usage predictions is 0.06920810009147811
The coefficients are [[0.00401626 0.00177797 0.02233777]]
0.19507694847521495
The mean of cap usage predictions is 0.06889892395919803
The coefficients are [[0.00154094 0.00206557 0.00421189]]
0.3589740191632802
The mean of cap usage predictions is 0.06824598634409673
The coefficients are [[0.00311984 0.00104923 0.05192208]]
0.22743401265916152
The mean of cap usage predictions is 0.06842889379639866
The coefficients are [[ 0.00345073 -0.01507612  0.08022642]]
0.24348392563334265
The mean of cap usage predictions is 0.06830814174222946
The coefficients are [[ 0.0048446   0.00010002 -0.01409452]]
0.15934991120529984
The mean of cap usage predictions is 0.06861614688189069
The coefficients are [[0.00124042 0.00067833 0.00949659]]
0.37355925815603075
The mean of cap usage predictions is 0.06932360433229681
The coefficients are [[0.00421157 0.00038957 0.02127455]]
0.19467746199040092
The mean of cap usage predicti

The coefficients are [[1.61748283e-02 8.50486053e-03 6.42573097e-05]]
0.30976552101924404
The mean of cap usage predictions is 0.0686929740498033
The coefficients are [[ 1.62342161e-02  8.46240360e-03 -7.09475542e-05]]
0.3097341056620103
The mean of cap usage predictions is 0.06868891214872444
The coefficients are [[0.00872199 0.00406362 0.00710952]]
0.407765156030619
The mean of cap usage predictions is 0.06929003661980696
The coefficients are [[0.01404865 0.00860274 0.00815278]]
0.31471557840477615
The mean of cap usage predictions is 0.06886492300949504
The coefficients are [[ 0.02250692 -0.0019798   0.00339536]]
0.4019031825729854
The mean of cap usage predictions is 0.06825339044023353
The coefficients are [[0.01284509 0.00780554 0.02326168]]
0.3215810783066173
The mean of cap usage predictions is 0.0686864104832821
The coefficients are [[0.01778731 0.00659693 0.01744837]]
0.31813122865897026
The mean of cap usage predictions is 0.06855005086171795
The coefficients are [[ 0.029869

0.40736358157515884
The mean of cap usage predictions is 0.06908020932267757
The coefficients are [[ 0.0326079  -0.00484678  0.00761709]]
0.3028903059007204
The mean of cap usage predictions is 0.06840281316418721
The coefficients are [[ 0.01955928 -0.0046088   0.00325578]]
0.4050797574037104
The mean of cap usage predictions is 0.06828372631056814
The coefficients are [[ 0.02558234 -0.01434163  0.04840668]]
0.33411429686769967
The mean of cap usage predictions is 0.06818263858526048
The coefficients are [[ 0.03141421 -0.0040108   0.02153441]]
0.31233368294615926
The mean of cap usage predictions is 0.06818801411489817
The coefficients are [[ 0.0337282  0.0003148 -0.0001015]]
0.29428631170278596
The mean of cap usage predictions is 0.06829698883877229
The coefficients are [[0.03374564 0.00025892 0.00193122]]
0.2942421376160532
The mean of cap usage predictions is 0.06828460108742475
The coefficients are [[0.01542023 0.00302748 0.00768784]]
0.40943740290294683
The mean of cap usage pred

...

The coefficients are [[0.00474737 0.00224917 0.00022291]]
0.14186847829256655
The mean of cap usage predictions is 0.06826102167959296
The coefficients are [[ 0.0046987   0.00234714 -0.00047816]]
0.14360285930466388
The mean of cap usage predictions is 0.06823359597851496
The coefficients are [[0.00472785 0.00225232 0.00516371]]
0.14209996370944444
The mean of cap usage predictions is 0.06821403520856441
The coefficients are [[-0.00149421  0.00270233  0.01168495]]
0.40060572342388867
The mean of cap usage predictions is 0.06885138071636639
The coefficients are [[0.00341835 0.00402872 0.03142208]]
0.20668458015932267
The mean of cap usage predictions is 0.06877493220567361
The coefficients are [[0.0020947  0.00237635 0.00427574]]
0.38495770880399716
The mean of cap usage predictions is 0.06831654519053411
The coefficients are [[0.00329139 0.00278265 0.06163768]]
0.2540184824964672
The mean of cap usage predictions is 0.06840387993189197
The coefficients are [[4.47788285e-03 2.46409411e-

0.2926740070927455
The mean of cap usage predictions is 0.06743120038892533
The coefficients are [[ 0.0153948   0.00385272 -0.00938937]]
0.23869670815751964
The mean of cap usage predictions is 0.06812985556192538
The coefficients are [[0.01642458 0.00288625 0.00468391]]
0.235281382027983
The mean of cap usage predictions is 0.06804323719546101
The coefficients are [[ 0.01556509  0.0031019  -0.00086416]]
0.22936313203921066
The mean of cap usage predictions is 0.06808103459194323
The coefficients are [[0.01506086 0.00323168 0.01017229]]
0.2248213464241251
The mean of cap usage predictions is 0.06804041977962265
The coefficients are [[0.00154244 0.00137315 0.00920107]]
0.37024916494626225
The mean of cap usage predictions is 0.0692372773306446
The coefficients are [[0.01390297 0.0031698  0.00652232]]
0.22674606910959738
The mean of cap usage predictions is 0.06827827880624536
The coefficients are [[0.00316098 0.00124722 0.00401166]]
0.3492543423857918
The mean of cap usage predictions i

The coefficients are [[ 0.00272665 -0.0209893   0.10082452]]
0.25954711455603685
The mean of cap usage predictions is 0.06745643259544892
The coefficients are [[-0.00026639  0.0015945   0.05471038]]
0.098044153312974
The mean of cap usage predictions is 0.06710465479829954
The coefficients are [[ 0.00257348 -0.00144102 -0.00077259]]
0.026539702691774858
The mean of cap usage predictions is 0.0668609461723247
The coefficients are [[ 0.00241107 -0.00158745  0.0130956 ]]
0.02354778715214545
The mean of cap usage predictions is 0.0667776787175859
The coefficients are [[0.0025132  0.00082546 0.01045586]]
0.39425110469355895
The mean of cap usage predictions is 0.06903289448462861
The coefficients are [[0.00443545 0.00056929 0.04185327]]
0.15200305082119736
The mean of cap usage predictions is 0.0680819141968455
The coefficients are [[ 0.00237611 -0.00013232  0.00472831]]
0.36455603291023364
The mean of cap usage predictions is 0.06779592040805116
The coefficients are [[ 0.00288873 -0.000319

The coefficients are [[ 0.00153463  0.0049326  -0.01709379]]
0.35006841713703296
The mean of cap usage predictions is 0.06813600080674977
The coefficients are [[0.00105119 0.00415087 0.01735098]]
0.3548587271587411
The mean of cap usage predictions is 0.06806033943848462
The coefficients are [[0.00207607 0.05327707 0.03603368]]
0.23578928409804067
The mean of cap usage predictions is 0.06806293755549499
The coefficients are [[ 0.00108063  0.0023505  -0.0006582 ]]
0.00635211867459029
The mean of cap usage predictions is 0.06693630767370784
The coefficients are [[0.00073757 0.00188832 0.01715876]]
0.005797016464765381
The mean of cap usage predictions is 0.06680803559168674
The coefficients are [[-0.0017524   0.00446158  0.01047195]]
0.3752436742607823
The mean of cap usage predictions is 0.06905916455848081
The coefficients are [[0.00023706 0.00616237 0.03391781]]
0.09614162909611224
The mean of cap usage predictions is 0.06794391833378897
The coefficients are [[-0.00210236  0.00329463 

The coefficients are [[0.00012232 0.03401103 0.05970307]]
0.19736546352176054
The mean of cap usage predictions is 0.06811177658589516
The coefficients are [[ 0.00017133  0.00517862 -0.01312272]]
0.34300429426932133
The mean of cap usage predictions is 0.06783035774357758
The coefficients are [[7.21692268e-05 4.38094426e-03 1.90451792e-02]]
0.35143881549894673
The mean of cap usage predictions is 0.06784007226097305
The coefficients are [[-0.00055177  0.06299474  0.04091467]]
0.22507650600717788
The mean of cap usage predictions is 0.06761789713973161
The coefficients are [[ 0.02001566  0.01184813 -0.01526359]]
0.38239212004148204
The mean of cap usage predictions is 0.06877064842362833
The coefficients are [[0.0260068  0.00704242 0.00174172]]
0.379514167917126
The mean of cap usage predictions is 0.06852660122484538
The coefficients are [[ 0.02739936  0.01039038 -0.00076861]]
0.37020353246352944
The mean of cap usage predictions is 0.06886261367775393
The coefficients are [[0.01996088